In [4]:
from base.api.market_data.classes.utilities import download_data
from base.api.market_data.classes.dataframe import EnhancedDataframe
from base.api.market_data.classes.strategies import TickerStrategy, Strategy
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [36]:
from backtesting import Backtest
from backtesting import Strategy


AAPL = df = EnhancedDataframe.populate_dataframe(yf.download("AAPL", period="2y", interval="1d"), "AAPL")
AAPL['Strategy Alpha'] = np.vectorize(TickerStrategy.ma_bol_rsi_signal)(
    AAPL['Close'], AAPL['MA50'], AAPL["BB_lower"], AAPL['RSI'])

AAPL['Strategy Beta'] = np.vectorize(TickerStrategy.r_ma20_ma50_signal)(
    AAPL['RSI'], AAPL['MA20'], AAPL['MA50'])

AAPL['Strategy Charlie'] = np.vectorize(
    TickerStrategy.r_sd_m_signal)(AAPL['RSI'], AAPL['STOCH_D'], AAPL['MACD_histogram'])


def entry_point(alpha, beta, charlie) -> bool:
    return alpha or beta or charlie


class StrategyTester(Strategy):

    def init(self):
        self.strat = self.I(
            entry_point, self.data['Strategy Alpha'], self.data['Strategy Beta'], self.data['Strategy Charlie'])
        print(list(filter(lambda x: x, self.strat)))
        print(len(list(filter(lambda x: x, self.strat))))

    
    def next(self):
        if self.strat:
            if not self.position:
                self.buy()
            
            


[*********************100%***********************]  1 of 1 completed


In [37]:


from backtesting import Backtest

bt = Backtest(AAPL, StrategyTester, cash=10_000, commission=.002)
stats = bt.run()

[True, True, True, True, True, True, True, True, True]
9


In [38]:
bt.plot()

Row(id='2680', ...)

In [14]:
stats

Start                     2020-12-09 00:00:00
End                       2022-08-19 00:00:00
Duration                    618 days 00:00:00
Exposure Time [%]                   85.948478
Equity Final [$]                145052.505722
Equity Peak [$]                 152577.742142
Return [%]                          45.052506
Buy & Hold Return [%]                40.84415
Return (Ann.) [%]                   24.545021
Volatility (Ann.) [%]               32.745562
Sharpe Ratio                         0.749568
Sortino Ratio                        1.381706
Calmar Ratio                         0.860251
Max. Drawdown [%]                  -28.532404
Avg. Drawdown [%]                   -4.238768
Max. Drawdown Duration      228 days 00:00:00
Avg. Drawdown Duration       27 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      45.076562
Worst Trade [%]                     45.076562
Avg. Trade [%]                    